In [2]:
import numpy as np
np.random.seed(1)

In [4]:

features=0
classes=0
samples=0
def modify_train_y(classes,train_y):
    new_train_y = []
    for x in train_y:
        current_y = []
        for i in range(classes):
            current_y.append(0)
        current_y[int(x)-1]=1
        new_train_y.append(current_y)
    return np.array(new_train_y).T


def data_loader(filename,isTrainData):
    # open data file
    file = open(filename,"r")


    # initialize
    i=0
    global features
    global classes
    global samples


    listx = []
    listy = []

    for line in file:

        fields = line.split()
        templist = []
        features = len(fields)-1
        for j in range(features):
                #print(fields[j])
            templist.append(float(fields[j]))

        listx.append(templist)
        listy.append(int(fields[features]))


    if isTrainData:
        samples = len(listx)
    if isTrainData:
        classes = len(set(listy))
    # convert into numpy array
    x = np.array(listx)
    y = np.array(listy)
    
    #print(x.shape, y.shape)
    
    return x,y


In [6]:
train_x,train_y =  data_loader("trainNN.txt",1)
train_x = train_x.T
train_y  = modify_train_y(classes,train_y)
print("Features: ",str(features), " Classes: ",str(classes)," Samples: ",str(samples))
print(train_x.shape)
print(train_y.shape)

Features:  4  Classes:  4  Samples:  500
(4, 500)
(4, 500)


In [8]:
test_x,test_y =  data_loader("trainNN.txt",0)
test_x = test_x.T

In [10]:
def sigmoid(z):
    return (1.0/(1.0+np.exp(-z)))
def derivative_sigmoid(z):
    return sigmoid(z)*(1-sigmoid(z))
def error(target,output):
    return  np.sum((target-output)*(target-output))/2.0
def derivative_error(target,output):
    return target-output



In [12]:
class Layer:
    def __init__(self,in_nodes,out_nodes):
        self.weight = np.random.randn(out_nodes,in_nodes)*.01
        self.bias = np.random.randn(out_nodes,1)*.01
        self.Z = None
        self.A = None

        self.dZ = None
        self.dA = None

        self.prev_A = None

    def forward(self,X):
        self.prev_A = X

        self.Z = np.dot(self.weight,X)+self.bias
        self.A = sigmoid(self.Z)

        return self.A


    def backward(self,delta,isLast , m ,learning_rate):

        if isLast:
            self.dA = self.A-delta
        else:
            self.dA = np.dot(self.weight, delta)

        self.dZ = self.dA * derivative_sigmoid(self.Z)

        self.dW = np.dot(self.dZ, self.prev_A.T)*(1.0/m)
        self.db =  np.sum(self.dZ, axis=1, keepdims=True)*(1.0/m)

        self.weight = self.weight  - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db

        return self.dZ





In [14]:
class Network:
    def __init__(self,features, classes, n_layers, size_layers):
        self.layers = []

        for i in range(1,len( size_layers)):
            self.layers.append(Layer(size_layers[i-1],size_layers[i]))
        
    def train(self,X,Y,epoch =100000, learning_rate = 0.0001):
        for i in range(epoch):
            A = X
            for layer in self.layers:
                A = layer.forward(A)
            
            delta = Y
            for layer in reversed(self.layers):
                delta = layer.backward(delta, layer == self.layers[-1] ,Y.shape[1],learning_rate)   

            if((i+1)%100==0):
                print('Iteration: ',str(i+1),' Error: ',error(self.layers[-1].A,Y))
                # for layer in self.layers:
                #     print('Weight: ',layer.weight)
                #     print('Bias: ',layer.bias)
        
        print('Train done!')

    def decide(self, X):
        A = X
        for layer in self.layers:
            Z = np.dot(layer.weight,A)+layer.bias
            A = sigmoid(Z)

        y_temp = A.T.tolist() 
        print(y_temp)
        y_hat  = []
        for row in y_temp:
            y_hat.append(row.index(max(row)) + 1)

        return y_hat

    def test(self,X,Y):
        m = len(Y)

        y_hat = self.decide(X)

        n_miss = 0.0
        for i in range(m):
            #print("Expected : %d    Found : %d" %(Y[i], y_hat[i]))
            print('Expected: ',Y[i],' Found: ',y_hat[i])
            if (Y[i] != y_hat[i]):
                n_miss += 1

        print("Test complete\n")

        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("Total number of test samples = %d" %(m))
        print("Correctly classified         = %d" %(m -n_miss))
        print("Misclassified                = %d" %(n_miss))

        accuracy = float(m - n_miss) / float(m)
        print("Accuracy                     = %f%%" %(accuracy * 100))
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")




In [16]:
network = Network(features,classes, 3, [features,4,4,classes])
network.train(train_x,train_y)
network.test(test_x,test_y)

Iteration:  100  Error:  249.52967260905902
Iteration:  200  Error:  249.37440047849014
Iteration:  300  Error:  249.21951754778507
Iteration:  400  Error:  249.06502307703693
Iteration:  500  Error:  248.91091632551243
Iteration:  600  Error:  248.75719655167293
Iteration:  700  Error:  248.60386301319465
Iteration:  800  Error:  248.45091496698956
Iteration:  900  Error:  248.29835166922558
Iteration:  1000  Error:  248.14617237534674
Iteration:  1100  Error:  247.99437634009345
Iteration:  1200  Error:  247.84296281752262
Iteration:  1300  Error:  247.6919310610271
Iteration:  1400  Error:  247.54128032335598
Iteration:  1500  Error:  247.39100985663376
Iteration:  1600  Error:  247.2411189123802
Iteration:  1700  Error:  247.09160674152935
Iteration:  1800  Error:  246.94247259444916
Iteration:  1900  Error:  246.79371572096034
Iteration:  2000  Error:  246.64533537035547
Iteration:  2100  Error:  246.4973307914179
Iteration:  2200  Error:  246.34970123244045
Iteration:  2300  Erro